## 8단계: 재귀에서 반복문으로

> 앞 단계에서는 Variable 클래스에 backward 메서드를 추가했습니다. \
이번에는 처리 효율을 개선하고 앞으로의 확장을 대비해 backward 메서드의 구현 방식을 바꿔보겠습니다.

### 8.1 현재의 Variable 클래스

In [ ]:
class Variable:
    ...  # 생략
    
    def backward(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()  # 이전 장에서는 Variable.backward를 재귀 구조로 구현했다.

### 8.2 반복문을 이용한 구현

재귀 구조를 반복문으로 바꿔보자.

In [2]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func
    
    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()  # 함수를 가져온다.
            x, y = f.input, f.output  # 함수의 입력과 출력을 가져온다.
            x.grad = f.backward(y.grad)  # backward 메서드를 호출한다.
            
            if x.creator is not None:
                funcs.append(x.creator)  # 하나 앞의 함수를 리스트에 추가한다.

### 8.3 동작 확인

개선된 Variable 클래스를 이용해보자.

In [ ]:
# 필요 모듈 정의

import numpy as np

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [4]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


7단계와 동일한 결과를 출력함을 알 수 있다.

반복문의 이점은 15단계에서 더 알 수 있다.

> 재귀는 함수 호출을 스택에 쌓으면서 처리하므로 보통 반복문으로 전환이 권장된다. \
그러나 요즘에는 컴퓨터 메모리도 넉넉하고, 꼬리 재귀(tail recursion) 기법을 이용하여 재귀를 반복문처럼 실행하기도 한다.